In [26]:
import json, sys, copy, os, igl
import numpy as np
import meshio

def is_inside(v, f, p):
    winds = igl.winding_number(v, f, p)
    mask = np.array(np.rint(winds), dtype=int)
    
    return np.array([e % 2 == 1 for e in mask])

def change_file_extension(filename, new_extension):
  base, ext = os.path.splitext(filename)
  return base + "." + new_extension if ext else filename + "." + new_extension

In [64]:
microstructure_repo_path = "/home/wxie/workspace/eFlesh/microstructure/microstructure_inflators"
matopt_repo_path = "/home/wxie/workspace/eFlesh/microstructure/matopt"
stitch_cells_cli = os.path.join(microstructure_repo_path, "build/isosurface_inflator/cut_cells_cli")
stitch_cells_cli = os.path.join(microstructure_repo_path, "build/isosurface_inflator/stitch_cells_cli")
only_cube_cells = False
input_surface = "fingers/finger_v4.stl"

In [65]:
E = 0.01
nu = 0.09
cell_size = 8
# out_path = str(E) + "_" + str(cell_size) + ".obj"
out_path = f"{input_surface[:-4]}_{E}_{cell_size}.obj_{only_cube_cells}.obj"
print(out_path)

m = meshio.read(input_surface)
input_surface = change_file_extension(input_surface, 'obj')
m.write(input_surface)
v = m.points.astype(float)
f = m.cells[0].data
bbox = [np.amin(v, axis=0), np.amax(v, axis=0)]

corner0 = list(map(int, np.ceil(bbox[0] / cell_size)))
# corner0 = list(map(int, np.ceil(bbox[0] / cell_size) - 1))
corner1 = list(map(int, np.floor(bbox[1] / cell_size)))

print(corner0, corner1)

# use return E for uniform Young's modulus
# use i,j,k to vary the Young's modulus through the dimensions
def young(i, j, k):
    return E
    print(k)
    if k == 0:
        return 0.001
    if k == 1:
        return 0.0015
    if k == 2:
        return 0.002
    if k == 3:  
        return 0.005
    if k == 4:
        return 0.005
    if k == 5:
        return 0.005
    if k == 6:
        return 0.005
    if k == 7:
        return 0.005
    if k == 8:
        return 0.005
    if k == 9:
        return 0.005
    
    

fingers/finger_v4_0.01_8.obj_False.obj
[0, 0, -3] [3, 2, 0]


In [66]:
sys.path.insert(0, f'{matopt_repo_path}/tools/material2geometry')
from material2geometry import Material2Geometry

In [67]:
mat2geo = Material2Geometry(in_path=f"{matopt_repo_path}/tools/material2geometry/0646_geo_1_coeffs.txt")

Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]
Res: [0.038461538461538464, 0.038461538461538464]
Width: [13, 13]


In [68]:
patterns = []
entry = {"params": [],
"symmetry": "Cubic",
"pattern": f"{microstructure_repo_path}/data/patterns/3D/reference_wires/pattern0646.wire",
"index": [0,0,0]}

for i in range(corner0[0], 1 + corner1[0]):
    for j in range(corner0[1], 1 + corner1[1]):
        for k in range(corner0[2], 1 + corner1[2]):
            geo_params = mat2geo.evaluate(nu, young(i, j, k))
            entry["params"] = geo_params
            entry["index"] = [i,j,k]
            patterns.append(copy.deepcopy(entry))

with open("data.json", 'w') as f:
    json.dump(patterns, f)

In [69]:
os.system(stitch_cells_cli + " -p data.json " +
           (("--surface " + input_surface) if not only_cube_cells else "") + 
           " --gridSize " + str(cell_size) + " -o " + out_path + " -r 50")
# print((stitch_cells_cli + " -p data.json " +
#            (("--surface " + input_surface) if not only_cube_cells else "") + 
#            " --gridSize " + str(cell_size) + " -o " + out_path + " -r 50"))

Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 2.08333 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 4.16667 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 6.25 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 8.33333 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 10.4167 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 12.5 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 14.5833 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 16.6667 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 18.75 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 20.8333 %
Checking printability
Inflation graph generated
Saved space 65.7472 %
Completed 22.9167 %
Checking printabil

0

In [70]:
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    for E in [1e-3, 2e-3, 5e-3, 1e-2]:
        print(f"Processing for c={c} and E={E}")
        print((np.array(mat2geo.evaluate(nu, E)[5:-4]) * c).tolist())

Processing for c=1 and E=0.001
[0.12500002236530952, 0.08039353577480896, 0.10304258195535204, 0.15000002683837113]
Processing for c=1 and E=0.002
[0.12500002321509995, 0.08246950095234824, 0.10661763958398165, 0.15000002785811953]
Processing for c=1 and E=0.005
[0.12500002462842463, 0.0886881045353454, 0.11693105096104062, 0.15000002955410885]
Processing for c=1 and E=0.01
[0.12500002371577018, 0.09894629144936973, 0.1327853472775627, 0.15000002845892332]
Processing for c=2 and E=0.001
[0.25000004473061904, 0.1607870715496179, 0.20608516391070408, 0.30000005367674226]
Processing for c=2 and E=0.002
[0.2500000464301999, 0.1649390019046965, 0.2132352791679633, 0.30000005571623906]
Processing for c=2 and E=0.005
[0.25000004925684927, 0.1773762090706908, 0.23386210192208123, 0.3000000591082177]
Processing for c=2 and E=0.01
[0.25000004743154036, 0.19789258289873946, 0.2655706945551254, 0.30000005691784665]
Processing for c=3 and E=0.001
[0.3750000670959286, 0.24118060732442687, 0.30912774

In [71]:
# To check printability of the geometry, we can create a DataFrame to summarize the minimum dimensions for different cell sizes and Young's moduli.Based on your printer nozzle size, we can determine if the geometry is printable or not.

import pandas as pd
import numpy as np

c_values = [1, 2, 3, 4, 5, 6, 7, 8, 16]
E_values = [1e-3, 2e-3, 5e-3, 1e-2, 2e-2]

df = pd.DataFrame(index=c_values, columns=E_values)

df.index.name = "Cell Size (c)"
df.columns.name = "Young's Modulus (E)"

for c in c_values:
    for E in E_values:
        arr = np.array(mat2geo.evaluate(nu, E)[5:-4]) * c
        min_dim = np.min(arr)        
        if min_dim < 0.4:
            df.loc[c, E] = "Unprintable"
        else:
            df.loc[c, E] = f"{min_dim:.4f}"

df

Young's Modulus (E),0.001,0.002,0.005,0.010,0.020
Cell Size (c),,,,,
1,Unprintable,Unprintable,Unprintable,Unprintable,Unprintable
2,Unprintable,Unprintable,Unprintable,Unprintable,Unprintable
3,Unprintable,Unprintable,Unprintable,Unprintable,Unprintable
4,Unprintable,Unprintable,Unprintable,Unprintable,0.4740
5,0.4020,0.4123,0.4434,0.4947,0.5925
6,0.4824,0.4948,0.5321,0.5937,0.7110
7,0.5628,0.5773,0.6208,0.6926,0.8295
8,0.6431,0.6598,0.7095,0.7916,0.9480
16,1.2863,1.3195,1.4190,1.5831,1.8960
